## start_and_spin_node()

In [1]:
import time
from rclpy.node import Node
from nimbro_utils.lazy import start_and_spin_node, stop_node

class TestNode(Node):
    def __init__(self, context=None):
        super().__init__("test_node", context=context)
        self.called = 0
        self.create_timer(1.0, self.callback)
    def callback(self):
        self.called += 1
        self.get_logger().info(f"Timer callback {self.called}")

node_env = start_and_spin_node(TestNode, blocking=False)
time.sleep(5)
stop_node(*node_env)

> Starting node 'TestNode'
2025-09-10 17:18:30.499 [INFO] [test_node]: Timer callback 1
2025-09-10 17:18:31.416 [INFO] [test_node]: Timer callback 2
2025-09-10 17:18:32.416 [INFO] [test_node]: Timer callback 3
2025-09-10 17:18:33.415 [INFO] [test_node]: Timer callback 4
2025-09-10 17:18:34.422 [INFO] [test_node]: Timer callback 5
> Stopped node 'TestNode'


## SelfShutdown

In [2]:
import time
from rclpy.node import Node
from rclpy.callback_groups import MutuallyExclusiveCallbackGroup
from nimbro_utils.lazy import start_and_spin_node, SelfShutdown

class TestNode(Node):
    def __init__(self, context=None):
        super().__init__("test_node", context=context)
        # raise SelfShutdown("Node is done.")
        self.called = 0
        # self.create_timer(0.1, self.callback)
        self.create_timer(0.1, self.callback, callback_group=MutuallyExclusiveCallbackGroup())
    def callback(self):
        self.called += 1
        if self.called < 4:
            self.get_logger().info(f"Timer callback {self.called}")
        elif self.called == 4:
            raise SelfShutdown("Node is done.")

start_and_spin_node(TestNode, blocking=True)

> Starting node 'TestNode'
2025-09-10 17:18:35.553 [INFO] [test_node]: Timer callback 1
2025-09-10 17:18:35.654 [INFO] [test_node]: Timer callback 2
2025-09-10 17:18:35.753 [INFO] [test_node]: Timer callback 3
> Node 'TestNode' shutting down: Node is done.


## block_until_future_complete()

In [3]:
import time
from rclpy.node import Node
from rclpy.callback_groups import MutuallyExclusiveCallbackGroup
from nimbro_utils.lazy import start_and_spin_node, SelfShutdown, block_until_future_complete
from std_srvs.srv import Trigger

class TestNode(Node):
    def __init__(self, context=None):
        super().__init__("test_node", context=context)
        self.srv = self.create_service(Trigger, "/trigger", self.service_callback, callback_group=MutuallyExclusiveCallbackGroup())
        self.create_timer(0.0, self.timer_callback, callback_group=MutuallyExclusiveCallbackGroup())
    def service_callback(self, request, response):
        self.get_logger().info("Received service call")
        response.success = True
        response.message = "Hello World!"
        return response
    def timer_callback(self):
        time.sleep(3.0)
        request = Trigger.Request()
        self.client = self.create_client(Trigger, "/trigger")
        self.get_logger().info("Calling service...")
        future = self.client.call_async(request)
        block_until_future_complete(self, future, timeout=None)
        if future.done():
            response = future.result()
            if response.success:
                raise SelfShutdown(response.message)
            else:
                raise RunTimeError(response.message)
        else:
            raise RunTimeError("Failed to receive response")

start_and_spin_node(TestNode, blocking=True)

> Starting node 'TestNode'
2025-09-10 17:18:40.163 [INFO] [test_node]: Calling service...
2025-09-10 17:18:40.214 [INFO] [test_node]: Received service call
> Node 'TestNode' shutting down: Hello World!


## create_throttled_subscription()

In [4]:
import time
from rclpy.node import Node
from rclpy.callback_groups import MutuallyExclusiveCallbackGroup
from nimbro_utils.lazy import start_and_spin_node, create_throttled_subscription
from std_msgs.msg import String

class TestNode(Node):
    def __init__(self, context=None):
        super().__init__("test_node", context=context)
        create_throttled_subscription(self, String, "/test_string", self.sub_callback, 1, throttle=0.4, callback_group=MutuallyExclusiveCallbackGroup())
        # self.create_subscription(String, "/test_string", self.sub_callback, 1, callback_group=MutuallyExclusiveCallbackGroup())
        self.pub = self.create_publisher(String, "/test_string", 1)
        self.called = 0
        self.create_timer(0.2, self.timer_callback)
    def sub_callback(self, msg):
        self.get_logger().info(f"Received message {msg.data}")
    def timer_callback(self):
        self.called += 1
        self.pub.publish(String(data=str(self.called)))
        if self.called == 10:
            raise SelfShutdown()

start_and_spin_node(TestNode, blocking=True)

> Starting node 'TestNode'
2025-09-10 17:18:41.267 [INFO] [test_node]: Received message 1
2025-09-10 17:18:41.867 [INFO] [test_node]: Received message 4
2025-09-10 17:18:42.265 [INFO] [test_node]: Received message 6
2025-09-10 17:18:42.666 [INFO] [test_node]: Received message 8
> Node 'TestNode' shutting down


## wait_for_message()

In [5]:
import time
from rclpy.node import Node
from rclpy.callback_groups import MutuallyExclusiveCallbackGroup
from nimbro_utils.lazy import start_and_spin_node, SelfShutdown, wait_for_message
from std_msgs.msg import String

class TestNode(Node):
    def __init__(self, context=None):
        super().__init__("test_node", context=context)
        self.create_timer(0.1, self.waiter_callback, callback_group=MutuallyExclusiveCallbackGroup())
        self.create_timer(0.0, self.pub_callback, callback_group=MutuallyExclusiveCallbackGroup())
    def waiter_callback(self):
        self.get_logger().info("Waiting for message...")
        success, message = wait_for_message(node=self, topic_name="/test_string", topic_type=String, qos=1, timeout=None)
        if success:
            raise SelfShutdown(f"Reiveived message: {message.data}")
        else:
            raise RunTimeError(message)
    def pub_callback(self):
        time.sleep(3.0)
        self.pub = self.create_publisher(String, "/test_string", 1)
        self.get_logger().info("Publishing message")
        self.pub.publish(String(data=str("Hello!")))

start_and_spin_node(TestNode, blocking=True)

> Starting node 'TestNode'
2025-09-10 17:18:44.208 [INFO] [test_node]: Waiting for message...
2025-09-10 17:18:47.109 [INFO] [test_node]: Publishing message
> Node 'TestNode' shutting down: Reiveived message: Hello!
